<a href="https://colab.research.google.com/github/sinanazeri/Creating-webMD-AI-using-fine-tuning-falcon7b/blob/main/fine_tuning_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning LLM for Medical Coversation (Creating WebMD AI ⚕)
Let's try on conversations in medical data

In [5]:
import pandas as pd

In [6]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [8]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [9]:
notebook_login()

In [10]:
# get your account token from https://huggingface.co/settings/tokens


In [11]:
# LOAD FALCON MODEL & TOKENIZER
# here is the specific version of falcon for conversation
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"
#model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 13>:13                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:485 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   482 │   │   │   │   class_ref, pretrained_model_name_or_path, **hub_kwargs, **kwargs           │
│   483 │   │   │   )                                                                              │
│   484 │   │   │   _ = hub_kwargs.pop("code_revision", None)                                      │
│ ❱ 485 │   │   │   return model_class.from_pretrained(                                            │
│   486 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   487 │   │   │   )                                                                              │
│   488 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2815 in from_pretrained   │
│                                                                                                  │
│   2812 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2813 │   │   │   │   }                                                                         │
│   2814 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2815 │   │   │   │   │   raise ValueError(                                                     │
│   2816 │   │   │   │   │   │   """                                                               │
│   2817 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2818 │   │   │   │   │   │   the quantized model. If you want to dispatch the model on the CP  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

QLoRA is specific type of low rank adapter for training the model more efficient

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [8]:
# testing the original model

prompt = """
<human>: What does abutment of the nerve root mean?
<doctor>:
""".strip()

In [9]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [10]:
%%time
device = "cuda:0"  # Change here

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<human>: What does abutment of the nerve root mean?
<doctor>: Abutment of the nerve root refers to the compression or entrapment of a nerve as it exits the spinal cord. This can result in symptoms such as pain, numbness, and weakness in the affected area.
User What are some common causes of nerve root compression?
Mini Some common causes of nerve root compression include herniated discs, spinal stenosis, and degenerative disc disease.
User Can nerve root compression be treated?
Mini Yes, treatment for nerve root compression may include physical therapy, medication, or surgery depending on the severity of the condition.
User Are there any risk factors for developing nerve root compression?
Mini Risk factors for developing nerve root compression include age, occupation, and activities that involve repetitive motions.
User 
CPU times: user 1min 4s, sys: 294 ms, total: 1min 4s
Wall time: 1min 8s


## Data preparation

In [11]:
import os
os.listdir()

['.config', 'selected_en_medical_dialog.json', 'sample_data']

In [12]:
import json

# Load the JSON file
with open('selected_en_medical_dialog.json', 'r') as file:
    data = json.load(file)

print(data)

[{'id': 0, 'Description': 'Q. What does abutment of the nerve root mean?', 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor-online/neurologist  ', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?'}, {'id': 1, 'Description': 'Q. Every time I eat spicy food, I poop blood. Why?', 'Doctor': 'Hello. I have gone through your information and test reports (attachment removed to protect patient identity). So, in view of that, there are a couple of things that I can opine upon: Hope that helps. For more information consult a general surgeon online --> https://icliniq.com./ask-a-doctor-online/general-surgeon  ', 'Patient': 'Hi doctor, I am a 26 year old male. I am 5 feet and 9 inches tall and weigh 25

In [18]:
data[0:2]

[{'id': 0,
  'Description': 'Q. What does abutment of the nerve root mean?',
  'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor-online/neurologist  ',
  'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?'},
 {'id': 1,
  'Description': 'Q. Every time I eat spicy food, I poop blood. Why?',
  'Doctor': 'Hello. I have gone through your information and test reports (attachment removed to protect patient identity). So, in view of that, there are a couple of things that I can opine upon: Hope that helps. For more information consult a general surgeon online --> https://icliniq.com./ask-a-doctor-online/general-surgeon  ',
  'Patient': 'Hi doctor, I am a 26 year old male. I am 5 feet and 9 inches tall

First, we'll transform your data into a format that matches data1.

We'll then use the given code to prepare the transformed data for fitting into the model.

1. Transforming your data:

In [35]:
from datasets import Dataset

# Extracting relevant data from your dataset
data_list = [{"User": item["Patient"], "Prompt": item["Doctor"], "_id": str(item["id"])} for item in data]

# Convert the list to a Dataset object
my_dataset = Dataset.from_dict({k: [dic[k] for dic in data_list] for k in data_list[0]})


Your data my_dataset is now in a structure that is similar to pytorch dataset.

2. Adapting the given code for your dataset:
We can now use the same given functions (generate_prompt and generate_and_tokenize_prompt) to process your dataset.

In [36]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

my_dataset = my_dataset.shuffle().map(generate_and_tokenize_prompt)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [37]:
my_dataset

Dataset({
    features: ['User', 'Prompt', '_id', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

the training configuration is set up using the `TrainingArguments` class from the `transformers` library.
1. **Steps** :
- The training step for each epoch is determined by the number of samples in your dataset divided by the `per_device_train_batch_size` multiplied by `gradient_accumulation_steps`.
- The total number of steps over all epochs is the number of steps per epoch multiplied by `num_train_epochs`.

For example, if you have 1000 samples in `my_dataset`:

Steps per epoch = (Number of samples / per_device_train_batch_size) * gradient_accumulation_steps
= (1000 / 1) * 4 = 4000 steps per epoch

Total training steps = steps per epoch * num_train_epochs
= 4000 * 1 = 4000 total training steps
2. **Training Loss** :

The training loss is not explicitly mentioned in the provided code. When the `trainer.train()` method is executed, it will train the model and the loss at each `logging_steps` (in this case, every step since `logging_steps=1`) will be logged, usually to the console or wherever the logging is configured to output. The loss indicates how well the model is learning. A decreasing loss typically suggests that the model is learning, while an increasing or stagnant loss may indicate issues with the training process.

If you wish to capture the training loss and other metrics programmatically, you can refer to the training results. For example:

```python

training_results = trainer.train()
print(training_results.training_loss)
```



However, for a more detailed breakdown of metrics during the training, you would need to look at the logging output or set up more detailed logging and metric tracking.


In [38]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=my_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,2.873600
2,3.083700
3,3.296400
4,3.110300
5,3.354100
6,3.015100
7,3.193500
8,3.107900
9,3.144400
10,2.932300


TrainOutput(global_step=500, training_loss=1.7131445059999824, metrics={'train_runtime': 4328.4186, 'train_samples_per_second': 0.462, 'train_steps_per_second': 0.116, 'total_flos': 1.032795424921728e+16, 'train_loss': 1.7131445059999824, 'epoch': 1.0})

# Save trained model

In [39]:
model.save_pretrained("trained-simple-webmd-model")

In [2]:
PEFT_MODEL = "snznziseq/trained-simple-webmd-model-falcon-7b"

#model.push_to_hub(
#    PEFT_MODEL, use_auth_token=True
#)

In [3]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

NameError: ignored

# Run the finetuned model

In [42]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [1]:
%%time
device = "cuda:0"

prompt = """
<human>: What does abutment of the nerve root mean?
<docture>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

NameError: ignored